<a href="https://colab.research.google.com/github/Nikhil768692/Email_Marketing_Case_Study/blob/main/Email_Marketing_Case_Study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: Setup & Data Loading


In [1]:
# Step 1: Install and import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load data from Google Drive
email_df = pd.read_csv("/content/drive/MyDrive/Email_Marketing_Case_Study project/email_table.csv")
opened_df = pd.read_csv("/content/drive/MyDrive/Email_Marketing_Case_Study project/email_opened_table.csv")
clicked_df = pd.read_csv("/content/drive/MyDrive/Email_Marketing_Case_Study project/link_clicked_table.csv")

# Quick look at the data
print("Email Table:")
display(email_df.head())

print("Opened Table:")
display(opened_df.head())

print("Clicked Table:")
display(clicked_df.head())


Email Table:


,email_id,email_text,email_version,hour,weekday,user_country,user_past_purchases
0,85120,short_email,personalized,2,Sunday,US,5
1,966622,long_email,personalized,12,Sunday,UK,2
2,777221,long_email,personalized,11,Wednesday,US,2
3,493711,short_email,generic,6,Monday,UK,1
4,106887,long_email,generic,14,Monday,US,6


Opened Table:


,email_id
0,284534
1,609056
2,220820
3,905936
4,164034


Clicked Table:


,email_id
0,609056
1,870980
2,935124
3,158501
4,177561


Step 2: Email Performance Metrics

In [2]:
# Total emails sent
total_emails = email_df.shape[0]

# Calculate open rate
open_rate = (opened_df.shape[0] / total_emails) * 100

# Calculate click rate
click_rate = (clicked_df.shape[0] / total_emails) * 100

print(f"Open Rate: {open_rate:.2f}%")
print(f"Click Rate: {click_rate:.2f}%")


Open Rate: 10.35%
Click Rate: 2.12%


Step 3: Feature Engineering

In [3]:
# Add open and click labels
email_df['opened'] = email_df['email_id'].isin(opened_df['email_id']).astype(int)
email_df['clicked'] = email_df['email_id'].isin(clicked_df['email_id']).astype(int)

# Encode categorical variables
df = email_df.copy()
df = pd.get_dummies(df, columns=['email_text', 'email_version', 'weekday', 'user_country'], drop_first=True)

# Features and target
X = df.drop(columns=['email_id', 'opened', 'clicked'])
y = df['clicked']


 Step 4: Model Training

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict
y_pred = clf.predict(X_test)

# Evaluation
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99     19547
           1       0.08      0.01      0.02       453

    accuracy                           0.97     20000
   macro avg       0.53      0.50      0.50     20000
weighted avg       0.96      0.97      0.97     20000

Confusion Matrix:
 [[19486    61]
 [  448     5]]


Step 5: Estimate CTR Improvement


In [5]:
# Predict probabilities for all data
df['predicted_prob'] = clf.predict_proba(X)[:, 1]

# Choose top X% most likely to click
threshold = df['predicted_prob'].quantile(0.9)
top_users = df[df['predicted_prob'] >= threshold]

# Simulated improved CTR
simulated_ctr = top_users['clicked'].mean() * 100
print(f"Simulated CTR on top 10% targeted users: {simulated_ctr:.2f}%")


Simulated CTR on top 10% targeted users: 15.73%


Step 6: Segment Analysis

In [6]:
# Example: Country-level analysis
country_clicks = email_df.groupby("user_country")['clicked'].agg(['count', 'sum'])
country_clicks['click_rate'] = (country_clicks['sum'] / country_clicks['count']) * 100

country_clicks.sort_values(by='click_rate', ascending=False)


,count,sum,click_rate
user_country,,,
UK,19939,492,2.467526
US,60099,1464,2.435981
ES,9967,83,0.832748
FR,9995,80,0.800400


Step 7: Final Insights & Conclusion

###  Final Insights

After analyzing the email marketing campaign data and building a predictive model, we found the following:

- **Open Rate**: ~10.35%
- **Click-Through Rate (CTR)**: ~2.12%
- Using a machine learning model (Random Forest), we were able to predict which users are more likely to click the link inside the email.
- By targeting the **top 10% most likely users**, we can simulate a significantly higher CTR, showing potential for optimization.

###  Key Learnings

- **Email Personalization** and **shorter content** tend to perform better in terms of user engagement.
- Users with **more past purchases** are more likely to engage with marketing emails.
- There are noticeable **differences in performance by country**, which suggests region-specific targeting could be effective.
- Timing (hour, weekday) also shows potential patterns and could be further optimized with A/B testing.

### 🚀 Recommendation

Instead of sending emails randomly, we recommend:
- Using this model to score and prioritize users.
- Focusing on the top percentile of users with high predicted probability to increase CTR.
- Running controlled A/B tests to continuously refine the targeting strategy.

---

This analysis demonstrates how data and machine learning can effectively guide email marketing strategies, resulting in better user engagement and more efficient campaign performance.
